In [1]:
#!pip install fastapi

In [2]:
import sys
sys.path.append('third_party/Matcha-TTS')


In [3]:
%pwd


'c:\\Users\\japan\\CosyVoice'

In [4]:
from cosyvoice.cli.cosyvoice import CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio

c:\Users\japan\miniconda3\envs\cosyvoice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


failed to import ttsfrd, use wetext instead


In [5]:
cosyvoice = CosyVoice2(r"C:\Users\japan\datasets\pretrained_models\CosyVoice2-0.5B-original", load_jit=False, load_trt=False, load_vllm=False, fp16=False)



Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
c:\Users\japan\miniconda3\envs\cosyvoice\lib\site-packages\lightning\fabric\__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
c:\Users\japan\miniconda3\envs\cosyvoice\lib\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-11-19 20:37:44,301 INFO input frame rate=25
c:\Users\japan\miniconda3\envs\cosyvoice\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torc

2025-11-19 20:38:02,345 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/revisions HTTP/1.1" 200 222
2025-11-19 20:38:03,203 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/repo/files?Revision=master&Recursive=True HTTP/1.1" 200 None
2025-11-19 20:38:04,138 DEBUG Starting new HTTPS connection (1): www.modelscope.cn:443


2025-11-19 20:38:07,647 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/revisions HTTP/1.1" 200 222
2025-11-19 20:38:08,091 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/repo/files?Revision=master&Recursive=True HTTP/1.1" 200 None


In [6]:
# cosyvoice_old = CosyVoice2(r"C:\Users\japan\datasets\pretrained_models\CosyVoice2-0.5B-original", load_jit=False, load_trt=False, load_vllm=False, fp16=False)


In [7]:
prompt_speech_16k = load_wav(r"C:\Users\japan\Workspaces\AI\NLP\Speech\ref.wav", 16000)
for i, j in enumerate(cosyvoice.inference_zero_shot('tự nhiên thấy mình thật là đẹp troai', 'cả hai bên hãy cố gắng hiểu cho nhau', prompt_speech_16k, stream=False)):
    torchaudio.save('zero_shot_new.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate) 


  0%|          | 0/1 [00:00<?, ?it/s]2025-11-19 20:38:59,683 INFO synthesis text tự nhiên thấy mình thật là đẹp troai.
c:\Users\japan\miniconda3\envs\cosyvoice\lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
2025-11-19 20:39:47,513 INFO yield speech len 8.8, rtf 5.435220138593153
100%|██████████| 1/1 [00:51<00:00, 51.97s/it]


In [8]:
# prompt_speech_16k = load_wav(r'C:\Users\japan\Workspace\AI_Engineer\Speech\ref.wav', 16000)
# for i, j in enumerate(cosyvoice_old.inference_zero_shot('tự nhiên thấy mình thật là đẹp troai', 'cả hai bên hãy cố gắng hiểu cho nhau', prompt_speech_16k, stream=False)):
#     torchaudio.save('zero_shot_old.wav'.format(i), j['tts_speech'], cosyvoice_old.sample_rate)


In [ ]:
# prompt_speech_16k = load_wav(audio_path, 16000)
# for i, j in enumerate(cosyvoice.inference_zero_shot(speech_script, prompt_script, prompt_speech_16k, stream=False)):
#     torchaudio.save(generated_file_name.format(i), j['tts_speech'], cosyvoice.sample_rate)


import os
import torchaudio
import torch

def generate_from_real_folder(
    real_folder: str,
    generated_folder: str,
    cosyvoice,
    sr: int = 16000
):
    """
    real_folder: path chứa .txt và .normalized.wav
    generated_folder: nơi lưu file speech sinh ra
    cosyvoice: model inference đã load sẵn
    sr: sample rate để load prompt speech
    """
    os.makedirs(generated_folder, exist_ok=True)

    # Thu thập các file txt
    real_files = [f for f in os.listdir(real_folder) if f.endswith(".wav")]

    print(f"Found {len(real_files)} real samples.")

    for wav_file in real_files[:400]:
        base_name = wav_file.replace(".wav", "")
        wav_path = os.path.join(real_folder, wav_file)
        txt_path = os.path.join(real_folder, base_name + ".txt")

        # Kiểm tra file .wav có tồn tại
        if not os.path.exists(wav_path):
            print(f"[WARN] Missing wav for {base_name}. Skipped.")
            continue

        # Đọc script
        with open(txt_path, "r", encoding="utf8") as f:
            speech_script = f.read().strip()
            prompt_script = speech_script  # có thể thay bằng prompt khác

        # Load prompt speech
        prompt_speech = load_wav(wav_path, sr)

        # Output filename
        out_path = os.path.join(generated_folder, base_name + ".wav")

        print(f"[Generating] {base_name} → {out_path}")

        # Tiến hành inference
        outputs = cosyvoice.inference_zero_shot(
            prompt_script,
            speech_script,
            prompt_speech,
            stream=False
        )

        # Model có thể trả nhiều chunk → chỉ lưu chunk cuối hoặc ghép lại
        all_audio = []
        for out in outputs:
            all_audio.append(out["tts_speech"])

        # Ghép lại toàn bộ
        generated_audio = torch.cat(all_audio, dim=-1)

        # Lưu file
        torchaudio.save(out_path, generated_audio, cosyvoice.sample_rate)

    print("Done generating all audio.")


In [10]:
#real_path = r"C:\Users\japan\datasets\Speech\metric-test" #vivoice
#generated_speech = r"C:\Users\japan\datasets\Speech\generated_speech\cosyvoice2"

#real_path = r"C:\Users\japan\Downloads\mcv-scripted-vi-v23.0\cv-corpus-23.0-2025-09-05\vi\outputs" #commonvoice vn

real_path = r"C:\Users\japan\datasets\Speech\viet_bud500"
generated_speech = r"C:\Users\japan\datasets\Speech\generated_speech\cosyvoice2_original"

generate_from_real_folder(real_path,generated_speech,cosyvoice)